In [1]:
import sys
sys.path.insert(0, '../')
from qhdopt import QHD
import numpy as np
import mpmath

ImportError: cannot import name 'BeforeValidator' from 'pydantic' (/Users/samuelkushnir/miniconda3/envs/QHDOPT-dev/lib/python3.9/site-packages/pydantic/__init__.cpython-39-darwin.so)

In [3]:
Q = np.array([[0.8931, 0.8673],
     [0.8673, 0.9079]])
b = np.array([0.8483, 0.8762])

NameError: name 'np' is not defined

In [1]:
# Set the desired precision (number of decimal places)
mpmath.mp.dps = 50  # for example, 50 decimal places

# Create the matrix A using numpy
A = np.random.rand(2, 2)

# Convert A to mpmath matrix and make it symmetric to ensure positive-definiteness
A_mpmath = mpmath.matrix(A)
A_mpmath = A_mpmath * A_mpmath.transpose()

# High precision x vector
x_mpmath = mpmath.matrix([0.34987313872918378423, 0.534563452331234876327])

# Calculate b using mpmath's dot product
b_mpmath = A_mpmath * x_mpmath

# Convert results back to numpy arrays for any further processing
A_np = np.array(A_mpmath.tolist(), dtype=np.float64)
b_np = np.array(b_mpmath.tolist(), dtype=np.float64)

NameError: name 'mpmath' is not defined

In [10]:
 1 + np.finfo(np.longdouble).eps

1.0000000000000002

In [16]:
def run_qhd(matrix, vector):
    vector = -1*vector
    model = QHD.QP(matrix, vector)
    model.dwave_setup(10, api_key_from_file='/Users/samuelkushnir/Documents/dwave_api_key.txt')
    model.optimize(verbose=0)
    return model.info["refined_minimizer"]

In [18]:
def IR_QHD(matrix, vector, IRprecision):
  ### Scaled Iterative Refinement for solving a linear system
  ### matrix: coefficient matrix of the linear system
  ### vector: right-hand side vector of the linear system
    nabla             = 1                             # Scaling factor
    rho               = 2                             # Incremental scaling
    d                 = len(matrix)                   # Dimension
    iteration         = 0
    x                 = np.zeros(d)                   # Solution
    r                 = vector                        # Residual
    con               = np.linalg.cond(matrix)             # Condition number
    res=[]
    while (np.linalg.norm(r)>IRprecision):
        # c = np.linalg.solve(matrix, nabla*r)          # Should be replaced by QHD c = QHD(matrix, nabla*r)
        c = run_qhd(matrix, nabla*r)

        x = x + (1/nabla)*c                           # Updating solution
        r = vector - np.dot(matrix, x)                # Calculating resisdual
        nabla = min(rho*nabla,1/(np.linalg.norm(r)))  # Updating scaling factor
        res.append(np.linalg.norm(r))
        iteration=iteration+1
    return res, c

print(IR_QHD(Q, b, 1e-4))

Received Task from D-Wave:
2024-01-21 22:49:33
Backend QPU Time: 0.02525277
Overhead Time: 4.025589276737671
([1.4865979986850008e-05], array([0.17483117, 0.79807703]))
